<a href="https://colab.research.google.com/github/Tiabet/BaekJoon/blob/main/KoGPT_Finetuning_without_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!pip install accelerate
!pip install datasets

In [2]:
!nvidia-smi

Fri Sep 29 08:53:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, DataCollatorWithPadding,TrainingArguments, Trainer, GPT2Config, TrainerCallback
from datasets import Dataset, load_dataset
import os
import numpy as np
import random

모델 호출

In [4]:
model_checkpoint = "skt/kogpt2-base-v2"

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_checkpoint, bos_token='<bos>', eos_token='<eos>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', sep_token = '<sep>', padding_side='left')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Load the existing model checkpoint
model_config = GPT2Config.from_pretrained(model_checkpoint)

# Add the new special token to the model's config
model_config.bos_token_id = tokenizer.bos_token_id
model_config.eos_token_id = tokenizer.eos_token_id
model_config.unk_token_id = tokenizer.unk_token_id
model_config.pad_token_id = tokenizer.pad_token_id
model_config.mask_token_id = tokenizer.mask_token_id
model_config.sep_token_id = tokenizer.sep_token_id  # Add this line

# Initialize the model with the updated configuration
model = GPT2LMHeadModel.from_pretrained(model_checkpoint, config=model_config)

model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 51203. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(51203, 768)

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(4)

데이터셋 구축

In [29]:
dataset = load_dataset('drive/MyDrive', data_files={
    'train': 'nikluge-sc-2023-train.jsonl',
    'validation': 'nikluge-sc-2023-dev.jsonl',
    'test': 'nikluge-sc-2023-test.jsonl'
})

In [30]:
sentence1 = []
sentence3 = []

for dictionary in dataset['train']['input']:
    sentence1.append(f"첫 번째 문장 : {dictionary['sentence1']}")
    sentence3.append(f"세 번째 문장 : {dictionary['sentence3']}")

dataset['train'] = dataset['train'].add_column('sentence1', sentence1)
dataset['train'] = dataset['train'].add_column('sentence3', sentence3)

In [31]:
sentence1 = []
sentence3 = []

for dictionary in dataset['validation']['input']:
    sentence1.append(f"첫 번째 문장 : {dictionary['sentence1']}")
    sentence3.append(f"세 번째 문장 : {dictionary['sentence3']}")

dataset['validation'] = dataset['validation'].add_column('sentence1', sentence1)
dataset['validation'] = dataset['validation'].add_column('sentence3', sentence3)

In [32]:
sentence1 = []
sentence3 = []

for dictionary in dataset['test']['input']:
    sentence1.append(f"첫 번째 문장 : {dictionary['sentence1']}")
    sentence3.append(f"세 번째 문장 : {dictionary['sentence3']}")

dataset['test'] = dataset['test'].add_column('sentence1', sentence1)
dataset['test'] = dataset['test'].add_column('sentence3', sentence3)

In [33]:
def tokenize_function(batch):
    sentences1 = batch['sentence1']
    sentences3 = batch['sentence3']
    outputs = batch['output']

    model_inputs = []

    for s1, s3, output in zip(sentences1, sentences3, outputs):
        # Convert list elements to strings
        sentence1_str = ''.join(map(str, s1))
        sentence3_str = ''.join(map(str, s3))
        output_str = ''.join(map(str,output))

        # Tokenize the input sentences
        inputs = tokenizer(
            '<bos>' + sentence1_str + '<sep>' + sentence3_str +\
            '<sep>' + '두 번째 문장 :'+ output_str + '<eos>',
            padding="max_length",
            max_length=64,  # Adjust this as needed
            truncation=True,
            return_attention_mask = True,
            return_tensors = 'pt'
        )

        # Append tokenized inputs and labels to the lists
        model_inputs.append(inputs)

    return {
        'input_ids': [item['input_ids'] for item in model_inputs],
        'attention_mask': [item['attention_mask'] for item in model_inputs],
        'labels': [item['input_ids'] for item in model_inputs],
    }

In [34]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/120140 [00:00<?, ? examples/s]

Map:   0%|          | 0/15017 [00:00<?, ? examples/s]

Map:   0%|          | 0/15018 [00:00<?, ? examples/s]

In [35]:
tokenized_datasets = tokenized_datasets.remove_columns(dataset["train"].column_names)

In [36]:
print(tokenizer.decode(tokenized_datasets['test']['input_ids'][0][0], skip_special_tokens=False))

<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><bos> 첫 번째 문장 : 서영이는 기차에 타서 자리에 앉았다.<sep> 세 번째 문장 : 서영이는 바로 승무원에게 기차표를 보여줬다.<sep> 두 번째 문장 :<eos>


In [37]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #동적 패딩, attention_mask, input_ids 등의 길이를 똑같이 맞춰줌

훈련

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [38]:
batch_size = 32
num_train_epochs = 3
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args  = TrainingArguments(
    output_dir=f"{model_name}-finetuned-koGPT-complete_story",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=num_train_epochs,
    logging_steps = logging_steps,
    push_to_hub=True,
)

In [39]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [40]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.251800,2.979005
2,3.816200,2.960697
3,3.281100,3.209206


TrainOutput(global_step=11265, training_loss=3.4496727098108555, metrics={'train_runtime': 6714.9437, 'train_samples_per_second': 53.674, 'train_steps_per_second': 1.678, 'total_flos': 1.177185927168e+16, 'train_loss': 3.4496727098108555, 'epoch': 3.0})

In [24]:
trainer.evaluate()

{'eval_loss': 2.805304765701294,
 'eval_runtime': 82.0153,
 'eval_samples_per_second': 183.1,
 'eval_steps_per_second': 5.731,
 'epoch': 3.0}

In [41]:
trainer.push_to_hub(commit_message="Training complete", tags="text generation")

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

'https://huggingface.co/Tiabet/kogpt2-base-v2-finetuned-koGPT-complete_story-finetuned-koGPT-complete_story/tree/main/'

테스트

In [42]:
model_checkpoint = "Tiabet/kogpt2-base-v2-finetuned-koGPT-complete_story"

model = GPT2LMHeadModel.from_pretrained(model_checkpoint)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_checkpoint, bos_token='<bos>', eos_token='<eos>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', sep_token = '<sep>', padding_side='left')

In [ ]:
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Assuming you have a preprocessed test dataset with input_ids and attention_mask
test_input_ids = tokenized_datasets['test']['input_ids']  # List of input_ids for your test dataset
test_attention_mask = tokenized_datasets['test']['attention_mask']  # List of attention_mask for your test dataset

generated_sentences = []

for input_ids, attention_mask in zip(test_input_ids, test_attention_mask):

    input_ids = torch.tensor(input_ids[0][:-1]).unsqueeze(0).to(device)
    attention_mask = torch.tensor(attention_mask[0][:-1]).unsqueeze(0).to(device)


    # Generate text using the model
    output = model.generate(input_ids,
                            do_sample=True,
                            attention_mask=attention_mask,
                            max_length=256,
                            #num_return_sequences=1,
                            top_k=30,
                            top_p=0.7,
                            temperature=0.9,
                            repetition_penalty=2.0,)  # Adjust parameters as needed

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_sentences.append(generated_text)


In [ ]:
len(generated_sentences)